In [1]:
!nvidia-smi

Thu Aug 28 15:35:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForSeq2SeqLM, AutoTokenizer

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/pegasus-xsum",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(
    "google/pegasus-xsum"
)
input_text = """Plants are remarkable organisms that produce their own food using a method called photosynthesis.
This process involves converting sunlight, carbon dioxide, and water into glucose, which provides energy for growth.
Plants play a crucial role in sustaining life on Earth by generating oxygen and serving as the foundation of most ecosystems."""
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = model.generate(**input_ids, cache_implementation="static")
print(tokenizer.decode(output[0], skip_special_tokens=True))

PackageNotFoundError: No package metadata was found for bitsandbytes

In [4]:
!pip install transformers[sentencepiece] datasets sacrebleu py72zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement py72zr (from versions: none)
ERROR: No matching distribution found for py72zr


In [5]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.55.4
Uninstalling transformers-4.55.4:
  Successfully uninstalled transformers-4.55.4
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 24.5 MB/s eta 0:00:00


In [5]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d8d2c85347327d087e4f30947f6b79696577b73734bafb6eeaeabb4b154b6559
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [6]:
from datasets import load_dataset,load_from_disk
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
# from datasets import load_metric
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
from datasets import load_dataset

ds = load_dataset("knkarthick/samsum")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [8]:
import zipfile
import tempfile
import os
from datasets import load_dataset
import json

def download_and_zip_dataset():
    # Load the dataset
    ds = load_dataset("knkarthick/samsum")

    # Create a temporary directory to store files
    with tempfile.TemporaryDirectory() as temp_dir:
        # Save each split as JSON files
        for split_name, dataset_split in ds.items():
            filename = os.path.join(temp_dir, f"{split_name}.json")
            dataset_split.to_json(filename)

        # Create zip file
        zip_filename = "samsum_dataset.zip"
        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(temp_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.basename(file_path))

        print(f"Dataset saved as {zip_filename}")
        return zip_filename

# Run the function
download_and_zip_dataset()

Creating json from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset saved as samsum_dataset.zip


'samsum_dataset.zip'

In [11]:
from transformers import AutoTokenizer, PegasusForConditionalGeneration
model=PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer=AutoTokenizer.from_pretrained("google/pegasus-xsum")
ARTICLE_TO_SUMMARIZE=(

    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs=tokenizer(ARTICLE_TO_SUMMARIZE,max_length=1024,return_tensors="pt")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [12]:
summary_ids=model.generate(inputs['input_ids'])
tokenizer.batch_decode(summary_ids,skip_special_tokens=True,clean_up_tokenization_spaces=False)[0]

"California's largest electricity provider has turned off power to hundreds of thousands of customers."

In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Fine Tuning

In [14]:
model="google/pegasus-cnn_dailymail"
tokenizer=AutoTokenizer.from_pretrained(model)
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [15]:
!wget
!unzip summarizer-data.zip

--2025-08-28 15:47:54--  https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/krishnaik06/datasets/refs/heads/main/summarizer-data.zip [following]
--2025-08-28 15:47:54--  https://raw.githubusercontent.com/krishnaik06/datasets/refs/heads/main/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  16.2MB/s    in 0.5s    

2025-08-28 15:47:56 (16.2 MB/s) - ‘summarizer-data.zip’ saved [7

In [17]:
dataset_samsum=load_from_disk("samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [18]:
split_lengths=[len(dataset_samsum[split]) for split in dataset_samsum]

In [20]:
print(f"Split lengths:{split_lengths}")
print(f"Features:{dataset_samsum['train'].column_names}")
print("\n Dialogues")
print(dataset_samsum['test'][2]['dialogue'])
print("\n Sumamary:")
print(dataset_samsum['test'][2]['summary'])


Split lengths:[14732, 819, 818]
Features:['id', 'dialogue', 'summary']

 Dialogues
Lenny: Babe, can you help me with something?
Bob: Sure, what's up?
Lenny: Which one should I pick?
Bob: Send me photos
Lenny:  <file_photo>
Lenny:  <file_photo>
Lenny:  <file_photo>
Bob: I like the first ones best
Lenny: But I already have purple trousers. Does it make sense to have two pairs?
Bob: I have four black pairs :D :D
Lenny: yeah, but shouldn't I pick a different color?
Bob: what matters is what you'll give you the most outfit options
Lenny: So I guess I'll buy the first or the third pair then
Bob: Pick the best quality then
Lenny: ur right, thx
Bob: no prob :)

 Sumamary:
Lenny can't decide which trousers to buy. Bob advised Lenny on that topic. Lenny goes with Bob's advice to pick the trousers that are of best quality.


## Perparing Dataset for Training For Sequence To Sequence Model

In [22]:
def convert_examples_to_features(example_batch):
  input_encodings=tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)
  return {
      'input_ids':input_encodings['input_ids'],
      'attention_mask':input_encodings['attention_mask'],
      'labels':target_encodings['input_ids']
  }

In [23]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [24]:
dataset_samsum_pt['test']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})

DataCollatorForSeq2Seq is a special data collator designed for sequenece-to-sequence models(e.g Pegasus, T5, BERT) that helps in preparing batches of data for training.

In [25]:
# Training
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [29]:
from transformers import TrainingArguments, Trainer
trainer_args=TrainingArguments(
    output_dir='pegasus-samsum',num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    eval_strategy='steps',eval_steps=500,save_steps=1e6,
    gradient_accumulation_steps=16
)

In [30]:
trainer=Trainer(model=model_pegasus,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,train_dataset=dataset_samsum_pt['test'],eval_dataset=dataset_samsum_pt['validation'])

/tmp/ipython-input-182184094.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(model=model_pegasus,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,train_dataset=dataset_samsum_pt['test'],eval_dataset=dataset_samsum_pt['validation'])


In [31]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alok-sg (alok-sg-indian-institute-of-information-technology-senap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=52, training_loss=2.978633917295016, metrics={'train_runtime': 515.0354, 'train_samples_per_second': 1.59, 'train_steps_per_second': 0.101, 'total_flos': 314017624350720.0, 'train_loss': 2.978633917295016, 'epoch': 1.0})

## Evaluation

In [50]:
def generate_batch_sized_chunks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i+batch_size]
def calculate_metric_on_test_ds(dataset,metric,model,tokenizer,batch_size=16,device=device,column_text='article',column_summary='highlights'):
  article_batches=list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches=list(generate_batch_sized_chunks(dataset[column_summary],batch_size))
  for article_batch, target_batch in tqdm(zip(article_batches,target_batches),total=len(article_batches)):
    inputs=tokenizer(article_batch,max_length=1024,truncation=True,padding="max_length",return_tensors="pt")
    summaries=model.generate(input_ids=inputs['input_ids'].to(device),
                             attention_mask=inputs['attention_mask'].to(device),
                             length_penalty=0.8,num_beams=8,max_length=128)
    decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]
    decoded_summaries=[d.replace(""," ") for d in decoded_summaries]

    metric.add_batch(predictions=decoded_summaries,references=target_batch)
  score=metric.compute()
  return score


In [51]:
!pip install evaluate

In [52]:
import evaluate
rouge_metric=evaluate.load('rouge')
rouge_names=['rouge1','rouge2','rougeL','rougeLsum']

In [53]:
rouge_metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value('string'), 'references': List(Value('string'))}, {'predictions': Value('string'), 'references': Value('string')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
 

In [54]:
score=calculate_metric_on_test_ds(dataset_samsum['test'][0:10],rouge_metric,trainer.model,tokenizer=tokenizer,batch_size=2,column_text="dialogue",column_summary='summary')

100%|██████████| 5/5 [00:25<00:00,  5.16s/it]


In [56]:
rouge_dict={rn:score[rn]for rn in rouge_names}
import pandas as pd
pd.DataFrame(rouge_dict,index=[f'pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.020873,0.0,0.020742,0.020666


In [57]:
### Interpreting Good VS Bad ROUGE Scores


In [58]:
## save model
model_pegasus.save_pretrained("pegasus-samsum-model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [61]:
## load model
tokenizer=AutoTokenizer.from_pretrained("/content/tokenizer")

In [63]:
from transformers import pipeline, set_seed

In [64]:
gen_kwargs={"length_penalty":0.8,"num_beams":8,"max_length":128}
sample_text=dataset_samsum["test"][0]["dialogue"]
reference=dataset_samsum['test'][0]['summary']
pipe=pipeline("summarization",model="pegasus-samsum-model",tokenizer=tokenizer)
print("Dialogue")
print(sample_text)

print("References")
print(reference)
print("Model Summary")
print(pipe(sample_text,**gen_kwargs)[0]['summary_text'])

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
References
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
Model Summary
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .


In [65]:
!zip -r wandb.zip /content/wandb
from google.colab import files
files.download('wandb.zip')

  adding: content/wandb/ (stored 0%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/ (stored 0%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/run-0jgn4aor.wandb (deflated 81%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/tmp/ (stored 0%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/tmp/code/ (stored 0%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/files/ (stored 0%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/files/wandb-metadata.json (deflated 41%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/files/requirements.txt (deflated 56%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/files/output.log (deflated 85%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/logs/ (stored 0%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/logs/debug-internal.log (deflated 69%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/logs/debug.log (deflated 82%)
  adding: content/wandb/run-20250828_160301-0jgn4aor/logs/debug-core.log (def

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
!zip -r tokenizer.zip /content/tokenizer
from google.colab import files
files.download('tokenizer.zip')

  adding: content/tokenizer/ (stored 0%)
  adding: content/tokenizer/tokenizer.json (deflated 78%)
  adding: content/tokenizer/spiece.model (deflated 50%)
  adding: content/tokenizer/tokenizer_config.json (deflated 94%)
  adding: content/tokenizer/special_tokens_map.json (deflated 82%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
!zip -r pegasus-samsum-model.zip /content/pegasus-samsum-model
from google.colab import files
files.download('pegasus-samsum-model.zip')

  adding: content/pegasus-samsum-model/ (stored 0%)
  adding: content/pegasus-samsum-model/generation_config.json (deflated 44%)
  adding: content/pegasus-samsum-model/model.safetensors (deflated 7%)
  adding: content/pegasus-samsum-model/config.json (deflated 61%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
!zip -r pegasus-samsum.zip /content/pegasus-samsum
from google.colab import files
files.download('pegasus-samsum.zip')

  adding: content/pegasus-samsum/ (stored 0%)
  adding: content/pegasus-samsum/checkpoint-52/ (stored 0%)
  adding: content/pegasus-samsum/checkpoint-52/generation_config.json (deflated 44%)
  adding: content/pegasus-samsum/checkpoint-52/model.safetensors (deflated 7%)
  adding: content/pegasus-samsum/checkpoint-52/training_args.bin (deflated 54%)
  adding: content/pegasus-samsum/checkpoint-52/rng_state.pth (deflated 26%)
  adding: content/pegasus-samsum/checkpoint-52/config.json (deflated 61%)
  adding: content/pegasus-samsum/checkpoint-52/trainer_state.json (deflated 64%)
  adding: content/pegasus-samsum/checkpoint-52/tokenizer.json (deflated 78%)
  adding: content/pegasus-samsum/checkpoint-52/spiece.model (deflated 50%)
  adding: content/pegasus-samsum/checkpoint-52/optimizer.pt (deflated 8%)
  adding: content/pegasus-samsum/checkpoint-52/tokenizer_config.json (deflated 94%)
  adding: content/pegasus-samsum/checkpoint-52/special_tokens_map.json (deflated 82%)
  adding: content/pegas

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>